In [ ]:
# !pip install pyvi

In [ ]:
from pyvi import ViTokenizer, ViPosTagger
import tensorflow as tf
import string
import os
from tqdm import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [ ]:
# %ls

data/                  scrape_news.ipynb           temp/
data_processing.ipynb  search_generate_text.ipynb  text_generator.ipynb
model/                 src/                        word_embedding.ipynb


## **Data Pre-Processing**

In [ ]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    # doc = doc.lower() # lower
    tokens = doc.split() # split into words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) # remove all punctuations
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

# print(clean_document('đường phố ở việt nam'))

In [ ]:
TRAIN_PATH = './data/vnexpress/processed_news'
DIRECTORY_LIST = os.listdir(TRAIN_PATH)
print(DIRECTORY_LIST)

['thoi-su', 'doi-song']


In [ ]:
INPUT_LENGTH = 50
sequences = []
count = 0

for d in DIRECTORY_LIST:
  if count == 0:
    count += 1
    continue
  FILE_LIST = os.listdir(TRAIN_PATH + f'/{d}')
  # print(FILE_LIST)
  for f in tqdm(FILE_LIST):
      f1 = open(file=TRAIN_PATH + f'/{d}' + f'/{f}', encoding='utf-8')
      doc = f1.read()
      tokens = clean_document(doc)
      # for i in range(INPUT_LENGTH + 1, len(tokens)):
      #     seq = tokens[i-INPUT_LENGTH-1:i]
      #     line = ' '.join(seq)
      #     sequences.append(line)
      for i in range(0, len(tokens), 51):
        seq = tokens[i:i+51]
        line = ' '.join(seq)
        sequences.append(line)
  break

100%|██████████| 41/41 [00:16<00:00,  2.41it/s]


In [ ]:
print('Total of senetences: ', len(sequences))
sequences[0:5]

Total of senetences:  453


['muốn cha_mẹ dạy con cách_trở_nên mạnh_mẽ nghiên_cứu thấy người tinh_thần dẻo_dai bền_bỉ thể_hiện một loạt đặc_điểm phẩm_chất tự_tin kiên_cường kiên_trì giúp họ trở_thành người hạnh_phúc thành_công hơn nhà nghiên_cứu phát_hiện trẻ_em người_lớn phát_triển trau_dồi dẻo_dai tinh_thần thời_gian bốn cha_mẹ dạy con_trẻ tinh_thần mạnh_mẽ trao quyền con nhà trị_liệu tâm_lý amy morin',
 'mỹ tự_tin động_lực bản_thân phần quan_trọng sức_mạnh tinh_thần có_nghĩa con bạn phụ_thuộc người khác thấy hài_lòng về bản_thân bạn giúp trẻ tự_tin về bản_thân bằng cách dạy chúng cụm lặp_đi lặp_lại nhắc_nhở chúng chịu trách_nhiệm về cách chúng suy_nghĩ cảm_nhận hành xử bất_kể người xung_quanh thế_nào amy gợi_ý câu khẩu_hiệu ngắn_gọn',
 'dễ nhớ tất_cả tôi làm cố_gắng hết_sức hành_động tự_tin tôi đủ tốt hôm_nay tôi chọn hạnh_phúc con thấy giá_trị làm khó_khăn khó_khăn chứng_kiến con mình thất_bại ở tuy_nhiên cha_mẹ con thấy chấp_nhận thử_thách dạy con chẳng_hạn cách chúng_ta chịu_đựng áp_lực dù thành_công hay th

In [ ]:
# filter several punctuations in 50803 sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ')
# create vocabulary (corpus) for each word in N sentences
tokenizer.fit_on_texts(sequences)

print(len(tokenizer.word_index))
tokenizer.word_index

In [ ]:
def create_input(text_data):
    input_sequences = []
    for line in text_data:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

sequences_digit = create_input(sequences)
print(sequences_digit[0:5])
print(len(sequences_digit))

[[46, 62], [46, 62, 268], [46, 62, 268, 5], [46, 62, 268, 5, 2043], [46, 62, 268, 5, 2043, 602]]
21675


In [ ]:
max_sequence_len = max([len(seq) for seq in sequences_digit])
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_sequence_len, padding='pre'))

## **Build model**

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

3812

In [ ]:
# separate into input and output
X = input_sequences[:,:-1]
y = input_sequences[:,-1]
# convert y to one-hot vector based on vocab (corpus)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [ ]:
print(len(X[0]))
print(len(X))
print(X)

50
21675
[[  0   0   0 ...   0   0  46]
 [  0   0   0 ...   0  46  62]
 [  0   0   0 ...  46  62 268]
 ...
 [  0   0   0 ...   5 167 184]
 [  0   0   0 ... 167 184  73]
 [  0   0   0 ... 184  73  74]]


In [ ]:
print('The length of a one-hot vector y: ', len(y[0]))
print('The collection of y vectors for all sentences', y)

The length of a one-hot vector y:  3812
The collection of y vectors for all sentences [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 50, input_length=50))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(512, return_sequences=True))
model.add(tf.keras.layers.LSTM(512))
model.add(tf.keras.layers.Dense(100, activation='relu'))
###
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax')) # can use linear for predict the index of word???
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            190600    
                                                                 
 batch_normalization_4 (Bat  (None, 50, 50)            200       
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 50, 512)           1153024   
                                                                 
 lstm_5 (LSTM)               (None, 512)               2099200   
                                                                 
 dense_9 (Dense)             (None, 100)               51300     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
summary = model.fit(X, y, batch_size=512, epochs=100, callbacks=[callback])

In [ ]:
# test_loss, test_acc = model.evaluate(X, y, verbose=2)

## **Dump to model files**

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator/model

/content/drive/My Drive/Colab Notebooks/Text_Generator/model


In [ ]:
import pickle
model.save('51_acc_language_model.h5')

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('sequences_digit.pkl', 'wb') as f:
    pickle.dump(sequences_digit, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
